In [1]:
import pandas as pd
import os
import zipfile
import csv
import subprocess

In [2]:
def read_csv_data(file_path):
    try:
        salary_data = pd.read_csv(file_path, low_memory=False)
        print('Data successfully loaded.')
        return salary_data
    except FileNotFoundError:
        print('Error: The specified file was not found.')
    except Exception as e:
        print(f'Error: {e}')

In [3]:
# Function to get employee details by name
def find_employee_by_name(name, data):
    try:
        employee_record = data[data['EmployeeName'].str.contains(name, case=False)]
        if employee_record.empty:
            print('No records found for the specified employee name.')
            return None
        return employee_record
    except Exception as e:
        print(f'Error: {e}')

In [4]:
# Data processing with dictionary
def convert_data_to_dict(data):
    try:
        salary_dict = data.drop_duplicates(subset='EmployeeName').set_index('EmployeeName').to_dict(orient='index')
        print('Data successfully processed into a dictionary.')
        return salary_dict
    except KeyError:
        print('Error: Ensure the data contains a column named "Name".')
    except Exception as e:
        print(f'Error: {e}')

In [5]:
# Export data to CSV and zip it
def save_data_as_csv_and_zip(data, folder_name):
    try:
        os.makedirs(folder_name, exist_ok=True)
        output_csv = os.path.join(folder_name, 'employee_data.csv')
        data.to_csv(output_csv, index=False)
        zip_path = folder_name + '.zip'
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            zipf.write(output_csv, os.path.basename(output_csv))
        print('Data successfully exported and zipped.')
    except Exception as e:
        print(f'Error: {e}')

In [6]:
# Unzip and display data using R
def extract_and_show_r(zip_path):
    try:
        r_script_path = os.path.join(os.getcwd(), 'unzip_and_display.R')
        if not os.path.exists(r_script_path):
            print(f'Error: R script not found at {r_script_path}')
            return
        subprocess.run(['Rscript', r_script_path, zip_path])
        print('Data successfully displayed using R.')
    except Exception as e:
        print(f'Error: {e}')

In [ ]:
def main():
    file_path = input('Enter the full path to the CSV file or leave blank to use the current directory: ').strip('"') or os.path.join(os.getcwd(), 'Total.csv')
    salary_data = read_csv_data(file_path)

    if salary_data is not None:
        # Process data
        employee_dict = convert_data_to_dict(salary_data)

        # Get specific employee details
        name = input('Enter employee name: ')
        employee_details = find_employee_by_name(name, salary_data)

        # Export and zip data
        if employee_details is not None:
            save_data_as_csv_and_zip(employee_details, 'Employee_Profile')

        # Unzip and display using R
        extract_and_show_r('Employee_Profile.zip')

if __name__ == '__main__':
    main()